In [1]:
import os
import torch
import torch.nn as nn
import albumentations as A
import warnings

from torchvision import models
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2

import required.hy_parameter
from required.utils import train, validate, save_model
from required.transform_list import * 
from required.dataset_temp import CustomDataset

os.environ['KMP_DUPLICATE_LIB_OK']='True'
warnings.filterwarnings('ignore')

In [2]:
# 윈도우 기반 그래픽 카드 엔비디아 사용하고 계신경우
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# m1 m2 칩셋 사용하시는분
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

cpu


In [3]:
net = models.__dict__["resnet18"](pretrained=True)  # pretrained = True 
net.fc = nn.Linear(512,5) # 5개의 자료 
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
criterion = nn.CrossEntropyLoss() # criterion
optim = torch.optim.Adam(net.parameters(), lr= required.hy_parameter.lr) # optimizer

In [5]:
# validation
val_transform = A.Compose(transform_val)  # val aug
val_dataset   = CustomDataset("./2022.12/12.19_d54_data/data/val", # val dataset 
                            transform= val_transform)  
val_loader   = DataLoader(val_dataset, shuffle=False,  # val dataloader
                        batch_size= required.hy_parameter.batch_size)

In [6]:
# criterion
criterion = nn.CrossEntropyLoss().to(device)

In [7]:
# optimizer
optim = torch.optim.Adam(net.parameters(), lr=required.hy_parameter.lr)

In [8]:
# model save dir
model_save_dir = "./model_save/"
os.makedirs(model_save_dir, exist_ok=True)

In [9]:
for index, transform_train in enumerate(transform_trains):
    # create train_transform directory
    folder_name_temp = transform_train.split("_")[0] + str(index)  # transform + str(index)
    model_save_dir= os.path.join("./model_save", folder_name_temp)
    os.makedirs(model_save_dir, exist_ok=True)

    train_transform = A.Compose(transform_train) # train aug
    train_dataset = CustomDataset("./data/train", # train dataset
                            transform= train_transform)  
    train_loader = DataLoader(train_dataset, shuffle=True,  # train dataloader
                            batch_size=required.hy_parameter.batch_size)

    print(f"\n>>> {folder_name_temp} start :\n")
    train(
        number_epoch = required.hy_parameter.epoch,  # 100 
        train_loader = train_loader,        # 위의 train loader
        val_loader   = val_loader,          # 위의 valid loader
        criterion    = criterion,           # Cross Entropy
        optimizer    = optim,               # Adam
        model        = net,                 # resnet18 + num_class 5로 수정
        save_dir     = model_save_dir,      # ./2022.12/12.22_d57_data/model_save\transform 0~4
        device       = device               # cpu
    )


>>> transform0 start :

start training...


0it [00:00, ?it/s]


TypeError: 'str' object is not callable